In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Esophageal_Cancer"
cohort = "GSE75241"

# Input paths
in_trait_dir = "../../input/GEO/Esophageal_Cancer"
in_cohort_dir = "../../input/GEO/Esophageal_Cancer/GSE75241"

# Output paths
out_data_file = "../../output/preprocess/Esophageal_Cancer/GSE75241.csv"
out_gene_data_file = "../../output/preprocess/Esophageal_Cancer/gene_data/GSE75241.csv"
out_clinical_data_file = "../../output/preprocess/Esophageal_Cancer/clinical_data/GSE75241.csv"
json_path = "../../output/preprocess/Esophageal_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profile of esophageal squamous cell carcinoma"
!Series_summary	"The goal was to identify the differently expressed genes between esophageal tumor and nonmalignant surrounding mucosa"
!Series_overall_design	"15 paired ESCC samples and matched nonmalignant mucosa were analyzed"
Sample Characteristics Dictionary:
{0: ['patient: 1', 'patient: 2', 'patient: 3', 'patient: 4', 'patient: 5', 'patient: 6', 'patient: 7', 'patient: 8', 'patient: 9', 'patient: 10', 'patient: 11', 'patient: 12', 'patient: 14', 'patient: 15', 'patient: 16'], 1: ['tissue: nonmalignant surrounding mucosa', 'tissue: esophageal tumor'], 2: [nan, 'tumor differentiation: poor', 'tumor differentiation: moderate', 'tumor differentiation: well']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data
# comparing esophageal squamous cell carcinoma tumor samples with non-malignant tissue
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Identify the rows in the sample characteristics dictionary for each variable

# For trait - tissue type (tumor vs normal)
trait_row = 1  # The key 1 has tumor vs non-malignant surrounding mucosa

# For age - not available in the provided data
age_row = None  # Age is not mentioned in the sample characteristics

# For gender - not available in the provided data
gender_row = None  # Gender is not mentioned in the sample characteristics

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """
    Convert tissue type to binary:
    0 - nonmalignant surrounding mucosa (control)
    1 - esophageal tumor (case)
    """
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "tumor" in value.lower():
        return 1
    elif "nonmalignant" in value.lower() or "surrounding mucosa" in value.lower():
        return 0
    else:
        return None

def convert_age(value):
    """Placeholder function for age conversion (not used)"""
    return None

def convert_gender(value):
    """Placeholder function for gender conversion (not used)"""
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save initial validation information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age, 
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical dataframe
    print("Clinical Data Preview:")
    print(preview_df(clinical_df))
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save clinical data to CSV
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{'GSM1946756': [0.0], 'GSM1946757': [1.0], 'GSM1946758': [0.0], 'GSM1946759': [1.0], 'GSM1946760': [0.0], 'GSM1946761': [1.0], 'GSM1946762': [0.0], 'GSM1946763': [1.0], 'GSM1946764': [0.0], 'GSM1946765': [1.0], 'GSM1946766': [0.0], 'GSM1946767': [1.0], 'GSM1946768': [0.0], 'GSM1946769': [1.0], 'GSM1946770': [0.0], 'GSM1946771': [1.0], 'GSM1946772': [0.0], 'GSM1946773': [1.0], 'GSM1946774': [0.0], 'GSM1946775': [1.0], 'GSM1946776': [0.0], 'GSM1946777': [1.0], 'GSM1946778': [0.0], 'GSM1946779': [1.0], 'GSM1946780': [0.0], 'GSM1946781': [1.0], 'GSM1946782': [0.0], 'GSM1946783': [1.0], 'GSM1946784': [0.0], 'GSM1946785': [1.0]}
Clinical data saved to ../../output/preprocess/Esophageal_Cancer/clinical_data/GSE75241.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 64
Header line: "ID_REF"	"GSM1946756"	"GSM1946757"	"GSM1946758"	"GSM1946759"	"GSM1946760"	"GSM1946761"	"GSM1946762"	"GSM1946763"	"GSM1946764"	"GSM1946765"	"GSM1946766"	"GSM1946767"	"GSM1946768"	"GSM1946769"	"GSM1946770"	"GSM1946771"	"GSM1946772"	"GSM1946773"	"GSM1946774"	"GSM1946775"	"GSM1946776"	"GSM1946777"	"GSM1946778"	"GSM1946779"	"GSM1946780"	"GSM1946781"	"GSM1946782"	"GSM1946783"	"GSM1946784"	"GSM1946785"
First data line: 2315554	8.17671	8.3064	8.2427	8.39671	8.51383	8.12902	8.30535	8.38525	7.97932	8.13759	8.328	8.35267	8.23582	8.12066	8.45462	7.89502	7.98993	8.2095	8.26696	7.91252	8.22498	8.40417	8.08198	8.26314	8.35753	8.09386	8.06862	7.72916	8.21028	8.10057
Index(['2315554', '2315633', '2315674', '2315739', '2315894', '2315918',
       '2315951', '2316218', '2316245', '2316379', '2316558', '2316605',
       '2316746', '2316905', '2316953', '2317246', '2317317', '2317434',
       '2317472', '2317512'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the identifiers in the gene expression data (2315554, 2315633, etc.)
# These appear to be numerical IDs rather than human gene symbols
# They are likely probe IDs from a microarray platform and need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE75241
Line 6: !Series_title = Gene expression profile of esophageal squamous cell carcinoma
Line 7: !Series_geo_accession = GSE75241
Line 8: !Series_status = Public on Jun 26 2019
Line 9: !Series_submission_date = Nov 20 2015
Line 10: !Series_last_update_date = Jan 13 2020
Line 11: !Series_pubmed_id = 29682174
Line 12: !Series_pubmed_id = 31901859
Line 13: !Series_summary = The goal was to identify the differently expressed genes between esophageal tumor and nonmalignant surrounding mucosa
Line 14: !Series_overall_design = 15 paired ESCC samples and matched nonmalignant mucosa were analyzed
Line 15: !Series_type = Expression profiling by array
Line 16: !Series_contributor = Pedro,P,Nicolau-Neto
Line 17: 


Gene annotation preview:
{'ID': [2315100, 2315106, 2315109, 2315111, 2315113], 'GB_LIST': ['NR_024005,NR_034090,NR_024004,AK093685', 'DQ786314', nan, nan, 'DQ786265'], 'SPOT_ID': ['chr1:11884-14409', 'chr1:14760-15198', 'chr1:19408-19712', 'chr1:25142-25532', 'chr1:27563-27813'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['11884', '14760', '19408', '25142', '27563'], 'RANGE_STOP': ['14409', '15198', '19712', '25532', '27813'], 'total_probes': ['20', '8', '4', '4', '4'], 'gene_assignment': ['NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 9 // 15q26.3 // 100288486 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// AK093685 // DDX11L2 // DEAD/H (As

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for gene IDs and gene symbols in the annotation dataframe
# Based on the preview, we need:
# - 'ID' column for probe IDs which matches the gene expression data index
# - 'gene_assignment' column which contains gene symbols

# First, let's parse the gene annotation properly 
gene_annotation = get_gene_annotation(soft_file)

# Display the columns to confirm we have the right ones
print("Gene annotation columns:", gene_annotation.columns.tolist())

# 2. Create a mapping dataframe with probe IDs and gene symbols
# Extract the ID column and gene_assignment column
if 'ID' in gene_annotation.columns and 'gene_assignment' in gene_annotation.columns:
    mapping_df = get_gene_mapping(gene_annotation, 'ID', 'gene_assignment')
    print(f"Mapping dataframe shape: {mapping_df.shape}")
    print("First few rows of mapping:")
    print(mapping_df.head())
    
    # 3. Apply the gene mapping to convert probe-level measurements to gene expression data
    # This handles the many-to-many relationships as described
    gene_data = apply_gene_mapping(gene_data, mapping_df)
    print(f"Gene expression data shape after mapping: {gene_data.shape}")
    print("First few rows of gene expression data:")
    print(gene_data.head())
    
    # Normalize gene symbols to official symbols and aggregate duplicate genes
    gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene expression data shape after normalization: {gene_data.shape}")
    
    # Save the gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
else:
    print("Error: Required columns not found in gene annotation dataframe")
    print("Available columns:", gene_annotation.columns.tolist())


Gene annotation columns: ['ID', 'GB_LIST', 'SPOT_ID', 'seqname', 'RANGE_GB', 'RANGE_STRAND', 'RANGE_START', 'RANGE_STOP', 'total_probes', 'gene_assignment', 'mrna_assignment', 'category']
Mapping dataframe shape: (316481, 2)
First few rows of mapping:
        ID                                               Gene
0  2315100  NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-As...
1  2315106                                                ---
2  2315109                                                ---
3  2315111                                                ---
4  2315113                                                ---


Gene expression data shape after mapping: (48895, 30)
First few rows of gene expression data:
      GSM1946756  GSM1946757  GSM1946758  GSM1946759  GSM1946760  GSM1946761  \
Gene                                                                           
A-     19.686233   21.078992   19.644611   19.255770   19.082776   19.978018   
A-2     3.041177    3.179220    3.144313    3.210930    3.121263    3.458567   
A-52    4.624367    4.627967    4.647400    4.688933    4.598800    4.648167   
A-E     1.734940    1.726010    1.729277    1.715336    1.663119    1.794793   
A-I     6.196227    6.294127    6.293147    6.598947    6.161653    6.351923   

      GSM1946762  GSM1946763  GSM1946764  GSM1946765  ...  GSM1946776  \
Gene                                                  ...               
A-     19.205867   19.183984   19.728438   19.648668  ...   19.475774   
A-2     3.038037    3.072120    3.068183    3.080643  ...    3.016570   
A-52    4.695467    4.721133    4.626467    4.796533 

Gene expression data saved to ../../output/preprocess/Esophageal_Cancer/gene_data/GSE75241.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note="Dataset contains gene expression data but lacks clear trait indicators for Duchenne Muscular Dystrophy status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note="Dataset contains gene expression data comparing Duchenne muscular dystrophy vs healthy samples."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (18418, 30)
First few genes with their expression values after normalization:
          GSM1946756  GSM1946757  GSM1946758  GSM1946759  GSM1946760  \
Gene                                                                   
A1BG        2.011265    1.946160    1.979805    2.054520    1.964573   
A1BG-AS1    2.011265    1.946160    1.979805    2.054520    1.964573   
A1CF        1.566023    1.530593    1.628517    1.542593    1.563090   
A2M         4.416360    4.980305    4.217160    4.178945    4.489790   
A2ML1       6.222100    5.579050    6.362350    5.534400    6.413550   

          GSM1946761  GSM1946762  GSM1946763  GSM1946764  GSM1946765  ...  \
Gene                                                                  ...   
A1BG        1.938865    1.984317    2.031433    1.959395     1.96234  ...   
A1BG-AS1    1.938865    1.984317    2.031433    1.959395     1.96234  ...   
A1CF        1.496433    1.624193    1.644417    1.559427     1.51756  ...   
A2M 

Normalized gene data saved to ../../output/preprocess/Esophageal_Cancer/gene_data/GSE75241.csv
Raw clinical data shape: (3, 31)
Clinical features:
                   GSM1946756  GSM1946757  GSM1946758  GSM1946759  GSM1946760  \
Esophageal_Cancer         0.0         1.0         0.0         1.0         0.0   

                   GSM1946761  GSM1946762  GSM1946763  GSM1946764  GSM1946765  \
Esophageal_Cancer         1.0         0.0         1.0         0.0         1.0   

                   ...  GSM1946776  GSM1946777  GSM1946778  GSM1946779  \
Esophageal_Cancer  ...         0.0         1.0         0.0         1.0   

                   GSM1946780  GSM1946781  GSM1946782  GSM1946783  GSM1946784  \
Esophageal_Cancer         0.0         1.0         0.0         1.0         0.0   

                   GSM1946785  
Esophageal_Cancer         1.0  

[1 rows x 30 columns]
Clinical features saved to ../../output/preprocess/Esophageal_Cancer/clinical_data/GSE75241.csv
Linked data shape: (30, 18419)
L

Data shape after handling missing values: (30, 18419)
For the feature 'Esophageal_Cancer', the least common label is '0.0' with 15 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Esophageal_Cancer' in this dataset is fine.



Linked data saved to ../../output/preprocess/Esophageal_Cancer/GSE75241.csv
